In [59]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  url="https://mermaid.ink/img/" + base64_string
  display(Image(url=url))

In [8]:
import decks_utils as du
import pandas as pd
import numpy as np
from itertools import combinations
import os
from zipfile import ZipFile
from datetime import datetime
import decks_utils as du
import sys

def extract_file(file_name):
    os.makedirs(f'tmp/{file_name}', exist_ok=True)
    with ZipFile(f'tmp/{file_name}.zip', 'r') as zipObj:
        zipObj.extractall(f'tmp/{file_name}')

    return f'tmp/{file_name}'

folder = extract_file("ringsdb")
decks = du.load_decks(folder)

decklist_df = du.cards_in_decks(du.count_cards_in_deck, decks)
columns = ['name', 'sphere', 'type', 'pack', 'quantity']
packs_in_collection = pd.read_csv("../data/packs.csv").head(33)
all_cards_rings_db = pd.read_csv("../data/cards_rings_db_info.csv")
cards_in_collection = packs_in_collection.merge(all_cards_rings_db, on='pack_name', how='left')
rename_dict = { 
    'sphere_name': 'sphere', 
    'type_name': 'type',
    'pack_name': 'pack'
}
cards_in_collection.rename(columns=rename_dict, inplace=True)
cards_in_collection = cards_in_collection[columns]
#adding Revised core set cards
revised_core_set = cards_in_collection[cards_in_collection.pack == 'Core Set'].copy()
revised_core_set['quantity'] = revised_core_set.apply(lambda x: 3 if x['type'] != 'Hero' else 1, axis=1)
revised_core_set['pack'] = 'Revised Core Set'

cards_in_collection = pd.concat([cards_in_collection, revised_core_set])

shared_cards = du.get_used_cards_in_collection(decklist_df, cards_in_collection)

cards_map = pd.pivot_table(
    shared_cards, values='quantity', 
    index=['name', 'sphere', 'type'],
    columns=['deck_name'], aggfunc=np.sum)\
.fillna(0)
cards_map.reset_index(inplace=True)
cards_in_collection = cards_in_collection.groupby(['name', 'sphere', 'type']).sum().reset_index().drop(columns=['pack'])
cards_in_collection.rename(columns={'quantity': 'in_collection'}, inplace=True)
cards_map = cards_map.merge(cards_in_collection, on=['name', 'sphere', 'type'], how='left')
cards_map.set_index(['name', 'sphere', 'type'], inplace=True)

def get_most_simultanious_deck(cards_map, i=None):
    deck_names = cards_map.drop(columns=['in_collection']).columns
    if i is None:
        i = len(deck_names)
    combs = list(combinations(deck_names, i))
    filtered_combs = list(filter(lambda x: (cards_map['in_collection'] >= cards_map[list(x)].sum(axis=1)).all(), combs))
    if len(filtered_combs):
        import math
        total_decks = len(deck_names)
        print("total decks", total_decks, "simultanious decks", i)
        print("max combinations", math.comb(total_decks, i))
        print("found combinations", len(filtered_combs))

        return filtered_combs

    return get_most_simultanious_deck(cards_map, i-1)

# get_most_simultanious_deck(cards_map, 5)
decklist_df.deck_name.unique()

array(['Anões', 'Rohan', 'Aragorn e seus crushs',
       'Arwen Is The Steward Now  (#RCO)', 'Dale (e rohan)',
       'The white counsel', 'Dúnedain e patrulheiros da TM', 'Silvan',
       'Starter for Dale (Single Core + Wilds of Rhovanion)', 'Hobbitses'],
      dtype=object)

In [126]:
def knapSack(W, wt, val, n, things=[]):
   # initial conditions
   if n == 0 or W.any() < 0 :
      return 0, things
   # If weight is higher than capacity then it is not included
   if (wt[n-1].all() > W.all()):
      return knapSack(W, wt, val, n-1, things)
   # return either nth item being included or not
   else:
      a = knapSack(W-wt[n-1], wt, val, n-1, things+[n-1])
      b = knapSack(W, wt, val, n-1, things)
      if val[n-1] + a[0] > b[0]:
         return val[n-1] + a[0], a[1]
      else:
         return b


def knapSack(W, wt, val, n):
    # initial conditions
   if n == 0 or W.any() < 0 :
       return 0
   # If weight is higher than capacity then it is not included
   if (wt[n-1].any() > W.any()):
      return knapSack(W, wt, val, n-1)
   # return either nth item being included or not
   else:
      return max(val[n-1] + knapSack(W-wt[n-1], wt, val, n-1),
         knapSack(W, wt, val, n-1))
       
W = cards_map['in_collection'].to_numpy()
wt = cards_map.drop(columns=['in_collection']).to_numpy().T
val = len(wt)*[1]
n = len(wt)
print(knapSack(W, wt, val, len(val)))

# val, W, wt

9


In [95]:
def knapSack(W, wt, val, n):
    K = [[0 for x in range(W + 1)] for x in range(n + 1)]
    # Build tаble K[][] in bоttоm uр mаnner
    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0  or  w == 0:
                K[i][w] = 0
            elif wt[i-1] <= w:
                K[i][w] = max(val[i-1]
                        + K[i-1][w-wt[i-1]],
                            K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
    return K[n][W]

# Driver code
W = cards_map['in_collection'].to_numpy()
wt = cards_map.drop(columns=['in_collection']).to_numpy().T
val = len(wt)*[1]
n = len(wt)


220


In [7]:
import decks_utils as du
import pdf_generator as pdf

decks = du.load_decks('./decks/')
decklist_df = du.cards_in_decks(du.count_cards_in_deck, decks)
all_cards = du.load_all_cards_df()
shared_cards = du.get_used_cards_in_collection(decklist_df, all_cards)
with open('decks/decks_new.tex', 'w') as f:
    txt = pdf.generate_latex(shared_cards)
    f.write(txt)
    f.close()
    
# from pylatex import Document, Package, Section, NoEscape

# txt = generate_latex(used_cards_df.groupby('deck_name'))
# geometry_options = {"tmargin": "1cm", "lmargin": "10cm"}
# doc = Document(geometry_options=geometry_options)
# doc.packages.append(Package('booktabs'))
# doc.append(txt)

# doc.generate_pdf('full', clean_tex=False)

In [58]:
from itertools import combinations
import decks_utils as du
import pdf_generator as pdf
import string


def remove_non_ascii(a_str):
    ascii_chars = set(string.printable)

    return ''.join(
        filter(lambda x: x in ascii_chars, a_str)
    )
    
decks = du.load_decks('./decks/')
decklist_df = du.cards_in_decks(du.count_cards_in_deck, decks)
all_cards = du.load_all_cards_df()
shared_cards = du.get_used_cards_in_collection(decklist_df, all_cards)
decks = shared_cards.drop_duplicates(subset=['deck_name'])
cards = shared_cards.drop_duplicates(subset=['name'])
decks = decks.deck_name.tolist()
cards = cards.name.tolist()
subgraph = 'graph LR;\n'
decks_declaration = {d:chr(65+i) for i, d in enumerate(decks)}
cards_declaration = {c:chr(97+i) for i, c in enumerate(cards)}
for d, id in cards_declaration.items():
    subgraph += f"\t{id}[\"{d}\"];\n"
for d, id in decks_declaration.items():
    subgraph += f"\t{id}[\"{d}\"];\n"

for card, decks_with_card in shared_cards.groupby('name'):
    card_id = cards_declaration[card]
    decks_with_card = shared_cards.drop_duplicates(subset=['deck_name'])
    decks_with_card = decks_with_card.deck_name.tolist()
    # decks = combinations(decks.deck_name.tolist(), 2)
    # lines = [f"\t{decks_declaration[d1]} --{card}--- {decks_declaration[d2]};" for d1, d2 in decks]
    lines = [f"\t{decks_declaration[d]} --- {card_id};"\
             for d in decks_with_card]
    subgraph += '\n'.join(lines) + '\n'
    # break
# print(remove_non_ascii(subgraph))
mm(remove_non_ascii(subgraph))